In [5]:
import os
import sys
import sqlite3
import numpy as np
import pandas as pd
import geopandas as gp
import plotly.express as px
import matplotlib.pyplot as plt
import scipy.stats as scs

import pyet
import pyeto
import spei

In [6]:
sys.path.append('/home/pooya/w/DroughtMonitoringIran/')

DATABASE_PATH = "./database/database.db"

### Monthly Data

In [7]:
conn = sqlite3.connect(DATABASE_PATH)

geoinfo = pd.read_sql(sql='SELECT * FROM ground_data_geoinfo', con=conn)

ground_data = pd.read_sql(sql='SELECT * FROM ground_data_monthly', con=conn)
ground_data['Date'] = pd.to_datetime(ground_data['Date'])
ground_data["Date"] = ground_data["Date"].dt.to_period("M").astype(str)

pet_data = pd.read_sql(sql='SELECT * FROM pet_monthly', con=conn)
pet_data['Date'] = pd.to_datetime(pet_data['Date'])
pet_data["Date"] = pet_data["Date"].dt.to_period("M").astype(str)

si_data = pd.read_sql(sql='SELECT * FROM si_monthly', con=conn)
si_data['Date'] = pd.to_datetime(si_data['Date'])
si_data["Date"] = si_data["Date"].dt.to_period("M").astype(str)

precip_data = pd.read_sql(sql='SELECT * FROM precip_monthly', con=conn)

di_data = pd.read_sql(sql='SELECT * FROM di_monthly', con=conn)

lst_data = pd.read_sql(sql='SELECT * FROM lst_monthly', con=conn)

pet_modis_data = pd.read_sql(sql='SELECT * FROM pet_monthly_modis', con=conn)

conn.close()

In [8]:
data = geoinfo.copy()

data = data\
    .merge(ground_data, on='Station_ID', how='outer')\
    .merge(pet_data, on=['Station_ID', 'Date'], how='outer')\
    .merge(si_data, on=['Station_ID', 'Date'], how='outer')\
    .merge(precip_data, on=['Station_ID', 'Date'], how='outer')\
    .merge(di_data, on=['Station_ID', 'Date'], how='outer')\
    .merge(lst_data, on=['Station_ID', 'Date'], how='outer')\
    .merge(pet_modis_data, on=['Station_ID', 'Date'], how='outer')\
    .query('Date >= "2006-01" and Date <= "2024-06"')\
    .reset_index(drop=True)

data

,Station_ID,Station_Name,Province,Station_Latitude,Station_Longitude,Station_Elevation,Date,Temp_Max,Temp_Min,Temp_Mean,...,LSTDay_MYD21C3,LSTNight_MOD21C3,LSTNight_MYD21C3,LSTDay,LSTNight,LST,TCIDay,TCINight,TCI,PET_MOD16A2GF
0,40709,Astara,Gilan,38.365,48.855,-21.1,2006-01,6.8,0.9,3.8,...,6.11,7.53,1.31,7.60,4.42,6.01,0.49,0.69,0.49,31.0875
1,40709,Astara,Gilan,38.365,48.855,-21.1,2006-02,9.2,3.2,6.2,...,6.77,5.39,3.19,9.27,4.29,6.78,0.64,0.84,0.80,70.3500
2,40709,Astara,Gilan,38.365,48.855,-21.1,2006-03,13.2,6.8,10.0,...,10.71,8.33,7.03,14.19,7.68,10.94,0.67,0.57,0.60,111.1375
3,40709,Astara,Gilan,38.365,48.855,-21.1,2006-04,16.5,10.9,13.7,...,14.87,9.91,8.23,19.98,9.07,14.53,0.42,0.67,0.51,120.2375
4,40709,Astara,Gilan,38.365,48.855,-21.1,2006-05,21.0,14.7,17.9,...,19.57,17.51,16.29,23.79,16.90,20.35,0.73,0.63,0.63,209.3000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5545,99361,Alasht,Mazandaran,36.071,52.843,1805.0,2024-02,7.5,-1.3,3.1,...,14.45,2.21,-1.47,7.28,0.37,3.83,0.47,0.48,0.48,NaN
5546,99361,Alasht,Mazandaran,36.071,52.843,1805.0,2024-03,12.1,1.8,6.9,...,22.47,3.73,-0.07,14.05,1.83,7.94,0.45,0.55,0.53,NaN
5547,99361,Alasht,Mazandaran,36.071,52.843,1805.0,2024-04,15.7,6.2,10.9,...,26.89,7.97,2.63,24.30,5.30,14.80,0.35,0.60,0.35,NaN
5548,99361,Alasht,Mazandaran,36.071,52.843,1805.0,2024-05,18.0,9.3,13.6,...,30.37,13.01,6.59,26.54,9.80,18.17,0.82,0.47,0.67,NaN


In [9]:
conn = sqlite3.connect(DATABASE_PATH)

data.to_sql('data', conn, if_exists='replace', index=False)
conn.commit()

conn.close()